In [36]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  7 11:57:13 2017

@author: saeed
"""
import numpy as np
import pandas as pd
from arch import arch_model
import math


stock_returns = pd.Series({'Google':np.random.normal(0, .2, 10),
                           'Apple':np.random.normal(0, .2, 10),
                           'Ebay':np.random.normal(0, .2, 10)})


returns=np.array(list(stock_returns.values))

initial_numberof_returns=returns[1].size # size of any of the security

#returns=returns[:,0:3]

#inititial condition setting esubt and q
esubt=[]
q_list=[]

cov_matrix=np.cov(returns)
diag=cov_matrix.diagonal()
diag_matrix=np.diag(np.diag(cov_matrix))
garch11 = arch_model(diag, p=1, q=1)
res = garch11.fit(update_freq=10)
alpha=res.params[2]
beta=res.params[3]

esubt.append(np.matmul(np.linalg.inv(diag_matrix),returns))
q_list.append(np.ones((3,3)))
pred_cov=[]


##############
#garch 1,1 forecasting
def simulate_GARCH(T, a0, a1, b1, sigma1):
    
    # Initialize our values
    X = np.ndarray(T)
    sigma = np.ndarray(T)
    sigma[0] = sigma1
    
    for t in range(1, T):
        # Draw the next x_t
        X[t - 1] = sigma[t - 1] * np.random.normal(0, 1)
        # Draw the next sigma_t
        sigma[t] = math.sqrt(a0 + b1 * sigma[t - 1]**2 + a1 * X[t - 1]**2)
        
    X[T - 1] = sigma[T - 1] * np.random.normal(0, 1)    
    
    return X, sigma

def dccGARCH(returns):    
    cov_matrix=np.cov(returns)
    diag=cov_matrix.diagonal()
    diag_matrix=np.diag(np.diag(cov_matrix))
    
    garch11 = arch_model(diag, p=1, q=1)
    res = garch11.fit(update_freq=10)
    alpha=res.params[2]
    beta=res.params[3]    
    
    esubt.append(np.matmul(np.linalg.inv(diag_matrix),returns))
    qdash=np.cov(esubt[-2])
    q_list.append(((1-alpha-beta)*qdash) + 
             (alpha*np.matmul(esubt[-2],esubt[-2].T)) + ((beta)*q_list[-1]))
    
    qstar=np.diag(np.diag(cov_matrix))
    qstar=np.sqrt(qstar)
    
    r=np.matmul(np.linalg.inv(qstar),q_list[-1],np.linalg.inv(qstar))
    
    pred_cov_temp=np.matmul(diag_matrix,r)
    pred_cov.append(np.matmul(pred_cov_temp,diag_matrix))
#     print(pred_cov[-1])

#performing garch11 to forecast the future returns which in turn is used for the dcc garch
iter=0
new_returns=[]

for n in returns:
    garch11 = arch_model(n, p=1, q=1)
    res = garch11.fit(update_freq=10)
    omega=res.params[1]
    alpha=res.params[2]
    beta=res.params[3]
    sig_ma=np.std(n)  # Assumption that I am making
    return_forecast,sigma_forecast=simulate_GARCH(5,omega,alpha,beta,sig_ma)
    n=np.append(n,return_forecast)
    print(return_forecast)
    print('\n')
    print(n)
    new_returns.append(n)
    print(new_returns)


returns=np.asanyarray(new_returns)
print(returns)

print(initial_numberof_returns," ",returns[1].size)    
for i in range(initial_numberof_returns,returns[1].size):
    dccGARCH(returns[:,0:i])


Optimization terminated successfully.    (Exit mode 0)
            Current function value: -10.7802542698
            Iterations: 8
            Function evaluations: 68
            Gradient evaluations: 8

 BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
Iteration:     10,   Func. Count:     67,   Neg. LLF: -1.4754635121174964
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1.47546839806
            Iterations: 11
            Function evaluations: 73
            Gradient evaluations: 11
[-0.04268826  0.10446308 -0.34682102  0.51599166  0.33661052]


[ 0.13312596 -0.38754741  0.22315998  0.04335996  0.13239837  0.3016526
 -0.01521972  0.14701028 -0.29305569  0.14976789 -0.04268826  0.10446308
 -0.34682102  0.51599166  0.33661052]
[array([ 0.13312596, -0.38754741,  0.22315998,  0.04335996,  0.13239837,
        0.3016526 , -0.01521972,  0.14701028, -0.29305569,  0.14976789,
       -0.04268826,  0.10446308, -0.34682102,  0.51599166,  0.33661052])]

 B

In [37]:
pred_cov[-1]

array([[ 0.18261911,  0.02548188,  0.00505598],
       [ 0.028947  ,  0.10111189, -0.00338786],
       [ 0.00695722, -0.00410378,  0.07259848]])

In [38]:
np.cov(returns)

array([[ 0.0645634 , -0.00037306, -0.00344418],
       [-0.00037306,  0.04671473, -0.0004891 ],
       [-0.00344418, -0.0004891 ,  0.03429668]])